In [5]:
import pandas as pd
import numpy as np
import os
import glob

In [6]:
# Cargar y unir los archivos CSV
# Definir la ruta donde están los archivos CSV
path_to_csvs = r"C:\Users\brand\ProyectoAnalisisCiber\data\raw\DatasetCICIDS2017\MachineLearningCVE"  # Directorio donde están los archivos CSV

# Se utiliza glob para encontrar todos los archivos CSV en el directorio
csv_files = glob.glob(os.path.join(path_to_csvs, "*.csv")) # Lista de todos los archivos CSV en el directorio

# Leer cada csv y guardarlo en una lista de dataframes
df_list = [] # Lista para almacenar los dataframes
for file_name in csv_files: # Iterar sobre cada archivo CSV
    df = pd.read_csv(file_name) # Leer el archivo CSV
    df_list.append(df) # Añadir el dataframe a la lista

# Concatenar todos los dataframes en uno solo
data = pd.concat(df_list, ignore_index = True) # Concatenar todos los dataframes en uno solo
print(f"Dimensiones del dataframe completo: {data.shape}") # Imprimir las dimensiones del dataframe completo

Dimensiones del dataframe completo: (2830743, 79)


In [7]:
# Limpieza y optimización inicial
data.columns = data.columns.str.strip() # Eliminar espacios en los nombres de las columnas

# El dataset CIC-IDS-2017 puede contener valores infinitos 
# Los reemplazamos con NaN (Not a Number) y luego eliminamos esas filas
data.replace([np.inf, -np.inf], np.nan, inplace = True)
data.dropna(inplace = True)
print(f"Nuevas dimensiones del DataFrame: {data.shape}")
data.isnull().sum().sum() # Verificar que no haya valores nulos

Nuevas dimensiones del DataFrame: (2827876, 79)


np.int64(0)

In [8]:
# Función para reducir el uso de memoria del DataFrame
def reduce_memory_usage(data):
    start_mem = data.memory_usage().sum() / 1024**2 # Memoria inicial en MB
    print(f"Memoria usada por el DataFrame: {start_mem:.2f} MB") 

    for col in data.columns: # Iterar sobre cada columna del DataFrame
        col_type = data[col].dtype # Tipo de dato de la columna

        if col_type != object and col_type.name != "category": # Si la columna no es de tipo object (cadena de texto)
            c_min = data[col].min() # Valor mínimo de la columna
            c_max = data[col].max() # Valor máximo de la columna

            if str(col_type)[:3] == "int": # Si la columna es de tipo entero
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max: # Verificar si cabe en int8
                    data[col] = data[col].astype(np.int8) # Convertir a int8
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max: # Verificar si cabe en int16
                    data[col] = data[col].astype(np.int16) # Convertir a int16
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max: # Verificar si cabe en int32
                    data[col] = data[col].astype(np.int32) # Convertir a int32
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max: # Verificar si cabe en int64
                    data[col] = data[col].astype(np.int64) # Convertir a int64
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max: # Verificar si cabe en float16
                    data[col] = data[col].astype(np.float16) # Convertir a float16
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max: # Verificar si cabe en float32
                    data[col] = data[col].astype(np.float32) # Convertir a float32
                else:
                    data[col] = data[col].astype(np.float64) # Convertir a float64
        else:
            data[col] = data[col].astype("category") # Convertir columnas de tipo object a category

    end_mem = data.memory_usage().sum() / 1024**2 # Memoria final en MB
    print(f"Memoria usada después de la optimización: {end_mem:.2f} MB") 
    print(f"Reducción de memoria: {100 * (start_mem - end_mem) / start_mem:.1f}%")
    return data 

data = reduce_memory_usage(data) # Aplicar la función de reducción de memoria
data.to_parquet(r"C:\Users\brand\ProyectoAnalisisCiber\data\processed\cic_ids_2017_optimized.parquet") # Guardar el DataFrame optimizado en formato Parquet

Memoria usada por el DataFrame: 1726.00 MB
Memoria usada después de la optimización: 655.34 MB
Reducción de memoria: 62.0%
